In [382]:
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils_OR.utils_OR import loadHdr

scene_name = 'scene0001_01'
# scene_name = 'scene0002_00'
frame_idx = 4

rendering_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/DatasetCreation/main_xml'
xml_path = '/newfoundland2/ruizhu/siggraphasia20dataset/code/Routine/scenes/xml'
layout_path = '/newfoundland2/ruizhu/siggraphasia20dataset/layoutMesh'

image_file = Path(rendering_path) / scene_name / ('im_%d.hdr'%frame_idx)
im_hdr = loadHdr(str(image_file))
fig = plt.figure()
ax = fig.gca()
ax.set_aspect("equal")
plt.imshow(im_hdr)
plt.show()

layout_file = Path(layout_path) / scene_name / ('%s_corners.npy'%scene_name)
corners = np.load(layout_file, allow_pickle=True)
print(corners.item())

layout_obj_file = Path(layout_path) / scene_name / 'uv_mapped.obj'
# layout_obj_file = Path(layout_path) / scene_name / ('%s_contour.obj'%scene_name)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


{'light_ctr': array([[5.9201, 3.8378, 3.    , 0.4   ]]), 'type': ['w', 'd', 'w'], 'coords': [array([[2.227 , 7.108 , 0.7257],
       [2.76  , 8.2028, 0.7257],
       [2.76  , 8.2028, 2.1603],
       [2.227 , 7.108 , 2.1603]]), array([[1.4655, 5.5439, 0.    ],
       [2.1127, 6.8734, 0.    ],
       [2.1127, 6.8734, 2.1603],
       [1.4655, 5.5439, 2.1603]]), array([[0.8564, 4.2926, 0.7257],
       [1.3513, 5.3092, 0.7257],
       [1.3513, 5.3092, 2.1603],
       [0.8564, 4.2926, 2.1603]])]}


In [710]:
from utils_OR.utils_OR_mesh import *
mesh = load_OR_mesh(layout_obj_file)
# mesh = mesh.dump()[0]
mesh = remove_top_down_faces(mesh)
v = np.array(mesh.vertices)
e = mesh.edges

%matplotlib widget
from utils.utils_rui import vis_axis
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_proj_type('ortho')
ax.set_aspect("auto")
vis_axis(ax)
v_pairs = v_pairs_from_v3d_e(v, e)
for v_pair in v_pairs:
    ax.plot3D(v_pair[0], v_pair[1], v_pair[2])

# find 2d floor contour
v_2d, e_2d = mesh_to_contour(mesh)
fig = plt.figure()
ax = fig.gca()
ax.set_aspect("equal")
v_pairs = v_pairs_from_v2d_e(v_2d, e_2d)
for v_pair in v_pairs:
    ax.plot(v_pair[0], v_pair[1])

# finding minimum 2d cuboid from contour
from utils_OR.utils_OR_mesh import minimum_bounding_rectangle, mesh_to_contour
layout_hull_2d = minimum_bounding_rectangle(v_2d)
hull_pair_idxes = [[0, 1], [1, 2], [2, 3], [3, 0]]
hull_v_pairs = [([layout_hull[idx[0]][0], layout_hull[idx[1]][0]], [layout_hull[idx[0]][1], layout_hull[idx[1]][1]]) for idx in hull_pair_idxes]
for v_pair in hull_v_pairs:
    ax.plot(v_pair[0], v_pair[1], 'b--')
plt.grid()

# simply mesh -> skeleton
v_skeleton, e_skeleton = mesh_to_skeleton(mesh)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_proj_type('ortho')
ax.set_aspect("auto")
vis_axis(ax)
v_pairs = v_pairs_from_v3d_e(v_skeleton, e_skeleton)
for v_pair in v_pairs:
    ax.plot3D(v_pair[0], v_pair[1], v_pair[2])

# 2d cuvboid hull -> 3d cuboid
room_height = 3.
layout_box_3d = np.hstack((np.vstack((layout_hull, layout_hull)), np.vstack((np.zeros((4, 1)), np.zeros((4, 1))+room_height))))    
vis_cube_plt(layout_box_3d, ax, 'b', linestyle='--')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [581]:
# transfer layout to world coordinates
transformFile = Path(xml_path) / scene_name / 'transform.dat'
# load transformations # writeShapeToXML.py L588
with open(str(transformFile), 'rb') as fIn:
    transforms = pickle.load(fIn )

transforms_layout = transforms[0]

v_skeleton_transform = transform_v(v_skeleton, transforms_layout)
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_proj_type('ortho')
ax.set_aspect("auto")
v_pairs = v_pairs_from_v3d_e(v_skeleton_transform, e_skeleton)
for v_pair in v_pairs:
    ax.plot3D(v_pair[0], v_pair[1], v_pair[2])
ax.view_init(elev=-71, azim=-65)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [770]:
# === Read cam intrinsics
from utils_OR.utils_OR_xml import get_XML_root, parse_XML_for_intrinsics
main_xml_file = Path(xml_path) / scene_name / 'main.xml'
root = get_XML_root(main_xml_file)

cam_K, intrinsics = parse_XML_for_intrinsics(root)


# === Read cam extrinsics

from utils_OR.utils_OR_cam import normalize, project_v, read_cam_params
cam_file = Path(xml_path) / scene_name / 'cam.txt'
cam_params = read_cam_params(cam_file)

np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

cam_param = cam_params[frame_idx-1]
origin, lookat, up = np.split(cam_param.T, 3, axis=1)
at_vector = lookat - origin
assert np.amax(np.abs(np.dot(at_vector.flatten(), up.flatten()))) < 1e-3 # two vector should be perpendicular

zaxis = normalize(lookat - origin)
# xaxis = normalize(np.cross(up.flatten(), zaxis.flatten()).reshape(3, 1))
xaxis = normalize(np.cross(up.T, zaxis.T).T)
yaxis = np.cross(zaxis.T, xaxis.T).T
# transformation = np.hstack([xaxis, yaxis, zaxis, np.zeros((3, 1))])
# transformation = np.vstack([transformation, \
#                             np.array([-np.dot(xaxis.flatten(), origin.flatten()), -np.dot(yaxis.flatten(), origin.flatten()), -np.dot(zaxis.flatten(), origin.flatten()), 1]).reshape(1, 4)])
# print(transformation)

# cam_R = np.eye(3, dtype=np.float)
# cam_t = np.zeros((3, 1), dtype=np.float)
# cam_identity_homo = np.vstack([np.hstack([cam_R, cam_t]), np.array([0., 0., 0., 1.]).reshape(1, 4)])

# v_proj_1 = project_v(v_skeleton_transform, cam_R, cam_t, cam_K)
# # print(v_proj)
# v_proj_2 = project_v_homo(v_skeleton_transform, cam_transformation_homo, cam_K)
# # print(v_proj)
# assert np.array_equal(v_proj_1, v_proj_2)

# shoube be the same:
# v_proj = project_v_homo(v_skeleton_transform, cam_identity_homo, cam_K)
# v_proj = project_v(v_skeleton_transform, cam_R, cam_t, cam_K)

# v_proj = project_v_homo(v_skeleton_transform, transformation, cam_K)
R_c = np.hstack([xaxis, yaxis, zaxis])
R_c = np.linalg.inv(R_c)
t_c = - R_c @ origin
v_proj = project_v(v_skeleton_transform, R_c, t_c, cam_K)

from PIL import Image, ImageDraw, ImageFont
img_map = Image.fromarray((im_hdr*255.).astype(np.uint8))
draw = ImageDraw.Draw(img_map)
    
v_tuples = v_xytuple_from_v2d_e(v_proj, e_skeleton) # [(x1y1, x2y2), ...]
v_tuples = [x for x in v_tuples if in_frame(x[0], width, height) or in_frame(x[1], width, height)]

for v_tuple in v_tuples:
    draw.line([tuple(v_tuple[0]), tuple(v_tuple[1])], width=2)
    
draw.line([(10, 10), (100, 10)], width=2)


layout_box_3d_transform = transform_v(layout_box_3d, transforms_layout)
layout_box_3d_transform_proj, front_flags = project_v(layout_box_3d_transform, R_c, t_c, cam_K, if_only_proj_front_v=False, if_return_front_flags=True)
from utils_OR.utils_OR import draw_projected_bdb3d
draw_projected_bdb3d(draw, layout_box_3d_transform_proj, front_flags=front_flags, color=(0, 0, 0))

plt.figure()
plt.imshow(np.array(img_map))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [399]:
print(cam_K)

print(R_c @ origin + t_c) # should equal to [0., 0., 0.] in cam coordinates

print(R_c @ lookat + t_c) # should equal to the z axis in cam coordinates

[[-577.8708    0.      320.    ]
 [   0.     -577.8708  240.    ]
 [   0.        0.        1.    ]]
[[0.]
 [0.]
 [0.]]
[[ 0.    ]
 [-0.    ]
 [ 1.0009]]


In [637]:
# draw layout, cam and world coordinates in 3D

from utils.utils_rui import Arrow3D
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_proj_type('ortho')
ax.set_aspect("auto")
v_pairs = v_pairs_from_v3d_e(v_skeleton, e_skeleton)
for v_pair in v_pairs:
    ax.plot3D(v_pair[0], v_pair[1], v_pair[2], 'gray')
v_pairs = v_pairs_from_v3d_e(v_skeleton_transform, e_skeleton)
for v_pair in v_pairs:
    ax.plot3D(v_pair[0], v_pair[1], v_pair[2])
ax.view_init(elev=-36, azim=89)
vis_axis(ax)

from utils.utils_rui import vis_axis_xyz
vis_axis_xyz(ax, xaxis.flatten(), yaxis.flatten(), zaxis.flatten(), origin.flatten(), suffix='_c')

a = Arrow3D([origin[0][0], lookat[0][0]*2-origin[0][0]], [origin[1][0], lookat[1][0]*2-origin[1][0]], [origin[2][0], lookat[2][0]*2-origin[2][0]], mutation_scale=20,
                lw=1, arrowstyle="->", color="k")
# a = Arrow3D([0., 1.,], [0., 1.,], [0., 1.,], mutation_scale=20,
#                 lw=1, arrowstyle="->", color="k")
ax.add_artist(a)
a_up = Arrow3D([origin[0][0], origin[0][0]+up[0][0]], [origin[1][0], origin[1][0]+up[1][0]], [origin[2][0], origin[2][0]+up[2][0]], mutation_scale=20,
                lw=1, arrowstyle="->", color="r")
ax.add_artist(a_up)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [774]:
# load object bboxes
root_uv_mapped = Path('/newfoundland2/ruizhu/siggraphasia20dataset/uv_mapped')
root_layoutMesh = Path('/newfoundland2/ruizhu/siggraphasia20dataset/layoutMesh')

from utils_OR.utils_OR_xml import parse_XML_for_shapes
shape_list = parse_XML_for_shapes(root, root_uv_mapped)

from utils_OR.utils_OR_mesh import loadMesh, computeBox, computeTransform
from utils_OR.utils_OR_transform import *

from tqdm import tqdm

# draw layout, cam and world coordinates in 3D
from utils.utils_rui import Arrow3D
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_proj_type('ortho')
ax.set_aspect("auto")
v_pairs = v_pairs_from_v3d_e(v_skeleton_transform, e_skeleton)
for v_pair in v_pairs:
    ax.plot3D(v_pair[0], v_pair[1], v_pair[2])
ax.view_init(elev=-36, azim=89)
vis_axis(ax)

# from utils.utils_rui import vis_axis_xyz
# vis_axis_xyz(ax, xaxis.flatten(), yaxis.flatten(), zaxis.flatten(), origin.flatten(), suffix='_c')

# a = Arrow3D([origin[0][0], lookat[0][0]*2-origin[0][0]], [origin[1][0], lookat[1][0]*2-origin[1][0]], [origin[2][0], lookat[2][0]*2-origin[2][0]], mutation_scale=20,
#                 lw=1, arrowstyle="->", color="k")
# # a = Arrow3D([0., 1.,], [0., 1.,], [0., 1.,], mutation_scale=20,
# #                 lw=1, arrowstyle="->", color="k")
# ax.add_artist(a)
# a_up = Arrow3D([origin[0][0], origin[0][0]+up[0][0]], [origin[1][0], origin[1][0]+up[1][0]], [origin[2][0], origin[2][0]+up[2][0]], mutation_scale=20,
#                 lw=1, arrowstyle="->", color="r")
# ax.add_artist(a_up)



from utils.utils_rui import vis_cube_plt
# vis_cube_plt(bverts_transformed, ax, 'r')
vertices_list = []
faces_list = []
num_vertices = 0
for shape_idx, shape in tqdm(enumerate(shape_list)):
    if 'container' in shape['filename']:
        continue

    if 'uv_mapped' in shape['filename']:
        obj_path = root_uv_mapped / shape['filename'].replace('../../../../../uv_mapped/', '')
    if 'layoutMesh' in shape['filename']:
        obj_path = root_layoutMesh / shape['filename'].replace('../../../../../layoutMesh/', '')

    vertices, faces = loadMesh(obj_path) # based on L430 of adjustObjectPoseCorrectChairs.py
    bverts, bfaces = computeBox(vertices )
#     vis_cube_plt(bverts, ax, 'r')
    
    bverts_transformed, transforms_converted_list = transform_with_transforms_xml_list(shape['transforms_list'], bverts)
    vertices_transformed, _ = transform_with_transforms_xml_list(shape['transforms_list'], vertices)
#     if shape_idx < len(transforms):
#         bverts_transformed = transform_with_transforms_dat_list(transforms[shape_idx], bverts)
#         vertices_transformed = transform_with_transforms_dat_list(transforms[shape_idx], vertices)
    if not(any(ext in shape['filename'] for ext in ['window', 'door', 'lamp'])):
#         vis_cube_plt(bverts_transformed, ax, 'r' if 'scene' not in shape['filename'] else 'b')
        if 'scene' not in shape['filename']:
            vis_cube_plt(bverts_transformed, ax, 'r')
    
    vertices_list.append(vertices_transformed)
#     vertices_list.append(vertices)
    
    faces_list.append(faces+num_vertices)
    num_vertices += vertices.shape[0]
    
#     print('======', shape_idx)
#     print(shape['id'], shape['filename'])
#     print(shape['transforms_list'])
#     print(transforms_converted_list)
#     if shape_idx < len(transforms):
#         print(transforms[shape_idx])

vis_cube_plt(layout_box_3d_transform, ax, 'b', '--')


# write to obj
vertices_combine = np.vstack(vertices_list)
faces_combine = np.vstack(faces_list)
from utils_OR.utils_OR_mesh import writeMesh

writeMesh('/home/ruizhu/Downloads/test_mesh.obj', vertices_combine, faces_combine)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

20it [00:03,  6.15it/s]
